In [6]:
from ultralytics import YOLO
import torch

# Check for GPU use
print('Use GPU:', torch.cuda.is_available())

if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("GPU cache cleared.\n")

# Training results: mAP50=89.2%, mAP70=87.9%
# Validation results: mAP50=89.6%, mAP70=88.5%

# val PyTorch model FP32
#model = YOLO('/home/heizung1/ultralytics_yolov8-obb_ob_kitti/ultralytics/kitti_bev_yolo/avoid_overfitting_best/weights/best.pt', task='obb', verbose=False) 
# val TensorRT FP32 dynamic
model = YOLO('/home/heizung1/ultralytics_yolov8-obb_ob_kitti/ultralytics/kitti_bev_yolo/avoid_overfitting_best/weights/fp32_batch=16/best.engine', task='obb', verbose=False) 
# val TensorRT FP16 dynamic
#model = YOLO('/home/heizung1/ultralytics_yolov8-obb_ob_kitti/ultralytics/kitti_bev_yolo/avoid_overfitting_best/weights/fp16_batch=16/best.engine', task='obb', verbose=False)  
# val TensorRT INT8 dynamic
#model = YOLO('/home/heizung1/ultralytics_yolov8-obb_ob_kitti/ultralytics/kitti_bev_yolo/avoid_overfitting_best/weights/int8_batch=16/best.engine', task='obb', verbose=False)

# Testing different models here
#model =  YOLO('/home/heizung1/ultralytics_yolov8-obb_ob_kitti/ultralytics/kitti_bev_yolo/run10_Adam_89.2_87.9/weights/best.engine', task='obb', verbose=False)

# Validate the model
metrics = model.val_kitti(data='/home/heizung1/ultralytics_yolov8-obb_ob_kitti/ultralytics/cfg/datasets/kitti_bev.yaml', imgsz=640, 
                    batch=16, save_json=False, conf=0.001, iou=0.5, max_det=20, half=False,
                    device='0', dnn=False, plots=False, rect=False, split='val', project=None, name=None,
                    verbose=True, save_txt=False, save_conf=False, save_crop=False, workers=8, augment=False,
                    agnostic_nms=False, single_cls=False) # rect=True increases mAP for Ped and Cyc

# Reproduce the validation values from post-training validation with official validation mode
#print('\nultralytics all classes mAP50: %.2f' % (metrics.box.map50 * 100)) # mAP50
#print('ultralytics all classes mAP70: %.2f' % (metrics.box.map70 * 100)) # mAP70
#print('ultralytics all classes mAP50-95: %.2f' % (metrics.box.map * 100)) # mAP50-95

# BEVDetNet metrics (adapted to 40-point interpolation like KITTI)
#print('\n')
#print(metrics.box.ap50_kitti(metrics.names)) # AP at IoU threshold of 0.5 for single classes
#print('\n')
#print(metrics.box.ap70_kitti(metrics.names)) # AP at IoU threshold of 0.7 for single classes
#print('\n')
#print(metrics.box.ap50_all_classes_average) # AP at IoU threshold of 0.5 for all classes
#print(metrics.box.ap70_all_classes_average) # AP at IoU threshold of 0.7 for all classes


Use GPU: True
GPU cache cleared.

class YOLO: current loaded model is: /home/heizung1/ultralytics_yolov8-obb_ob_kitti/ultralytics/kitti_bev_yolo/avoid_overfitting_best/weights/fp32_batch=16/best.engine
class YOLO: current task is: obb
Ultralytics 8.3.55 🚀 Python-3.12.8 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3080, 9898MiB)
Loading /home/heizung1/ultralytics_yolov8-obb_ob_kitti/ultralytics/kitti_bev_yolo/avoid_overfitting_best/weights/fp32_batch=16/best.engine for TensorRT inference...
[06/15/2025-17:36:16] [TRT] [I] The logger passed into createInferRuntime differs from one already provided for an existing builder, runtime, or refitter. Uses of the global logger, returned by nvinfer1::getLogger(), will return the existing value.
[06/15/2025-17:36:16] [TRT] [I] Loaded engine size: 49 MiB
[06/15/2025-17:36:16] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in IExecutionContext creation: CPU +0, GPU +581, now: CPU 1, GPU 1877 (MiB)


val: Scanning /home/heizung1/ultralytics_yolov8-obb_ob_kitti/ultralytics/cfg/datasets/kitti/labels/val.cache... 3699 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3699/3699 [00:00<?, ?it/s]


before custom table


                 Class     Images  Instances      Box(P          R       AP50      AP70): 100%|██████████| 232/232 [00:15<00:00, 14.52it/s]


--- Check all difficulty categories ---
                   all       3699      16907     0.6457      0.569     0.5994     0.5621
                   Car       3343      13783       0.86     0.8561     0.8984     0.8862
                   Ped        817       2267     0.5019     0.3939     0.3938     0.3522
                   Cyc        602        857     0.5754     0.4569     0.5059     0.4479

Distribution of difficulty levels across all objects:
            Difficulty        All       Easy   Moderate       Hard    Unknown
                            16907       4398       5929       3599       2981

Objects per class and difficulty level:
                 Class       Easy   Moderate       Hard    Unknown
                   Car       2973       5063       3105       2642
                   Ped       1136        603        439         89
                   Cyc        289        263         55        250
after custom table
Speed: 0.1ms preprocess, 1.8ms inference, 0.0ms loss, 1.0ms post

## Get desired result.csv value

In [ ]:
import pandas as pd
import os


input_file = '/home/heizung1/ultralytics_yolov8-obb_ob_kitti/ultralytics/kitti_bev_yolo/avoid_overfitting/results.csv'

output_prefix = '_custom'
output_dir = os.path.dirname(input_file)  # Verzeichnis der Eingabedatei
output_filename = os.path.basename(input_file)  # Dateiname der Eingabedatei
output_file = os.path.join(output_dir, f"{os.path.splitext(output_filename)[0]}{output_prefix}.csv")

df = pd.read_csv(input_file)

selected_columns = [
    "epoch",
    "train/box_loss",
    "train/cls_loss",
    "train/dfl_loss",
    'val/box_loss',
    'val/cls_loss',
    'val/dfl_loss'
]

filtered_df = df[selected_columns]

filtered_df.to_csv(output_file, index=False)
